In [1]:
from dataloader import *
%matplotlib inline
import sys
sys.path.append("./coco-caption")
import matplotlib.pyplot as plt
import skimage.io as io
import pylab
pylab.rcParams['figure.figsize'] = (10.0, 8.0)
from json import encoder
encoder.FLOAT_REPR = lambda o: format(o, '.3f')
from dataloader import get_dataset_configuration, load_all_captions_flickr, load_all_captions_coco
import glob
from IPython.display import display, clear_output
from collections import deque
from ipywidgets import HBox, Output, Button, widgets
from prettytable import PrettyTable
from sklearn.feature_extraction.text import CountVectorizer

In [81]:
def load_images_flickr(dataset_configuration):
    """
    Method to map images ids to pictures for data in Flickr structure

    Parameters
    ----------
    images_dir: str
        Path to the directory with all images from  Flickr type dataset
    Returns
    -------
    all_images_mapping: dict->
        paths to all images

    """
    
    all_captions = load_all_captions_flickr(dataset_configuration["captions_file_path"])

    images_dir = dataset_configuration["images_dir"]
    train_images = set(open(dataset_configuration[
                    "train_images_names_file_path"], 'r').read().strip().split('\n'))
    
    
    # add global paths to the all images in images_dir directory
    all_images = glob.glob(images_dir + '*.jpg')
    all_images_mapping = dict()
    train_images_mapping = dict()
    for i in all_images:  # img is list of full path names of all images
        image_name = i.split("/")[-1]
        image_id = image_name.split(".")[0]
        all_images_mapping[image_id] = i 
        if image_name in train_images and image_id in all_captions.keys():
            train_images_mapping[image_id] = all_captions[image_id]
    return all_images_mapping, train_images_mapping




def load_images_coco(configuration):
    """
    Method to map images ids to pictures for data in COCO structure

    Parameters
    ----------
    configuration
        Configuration of the dataset, with paths to the images and
         datasets specific files like file that mapps images with image id
    Returns
    -------
    all_images_mapping: dict->
        paths to all images from coco type data identidied by image ids

    """
    all_captions = load_all_captions_coco(configuration["captions_file_path"])
    
    file_with_images_def = configuration["images_names_file_path"]
    images_folder = configuration["images_dir"]
    info = json.load(open(file_with_images_def))
    all_images_mapping = dict()
    train_images_mapping = dict()
    for ix in range(len(info['images'])):
        img = info['images'][ix]
        image_filename = img['file_path'].rsplit(".", 1)[0]
        #create global path to the image by users directory
        file_path = images_folder + "/" + img['file_path']

        if image_filename.find("/") != -1:
            image_filename = img['file_path'].rsplit("/", 1)[1].rsplit(".", 1)[0]
        #define data splits
        if img['split'] in ['train','val', 'test', 'restval']:
            all_images_mapping[image_filename] = file_path
            print(image_filename)
            print(all_captions[image_filename])
            if img['split'] == 'train' and image_filename in all_captions.keys():
                    train_images_mapping[image_filename] = all_captions[image_filename]
            all_images_mapping[image_filename] = file_path

    return all_images_mapping, train_images_mapping

def get_images_for_split(dataset_name):
    # Load dataset configuration, by the name of the dataset assigned for training/testing
    train_dataset_configuration = get_dataset_configuration(dataset_name)
    # Therefore Flickr and COCO have different file and data structures, to show captions and split of data
    # different methods for loading captions and images are used.
    # Datasets Flickr30k, COCO2017, COCO2014 have the same strucutre of files with captions and split informations.
    if train_dataset_configuration["data_name"] in ["flickr30k", "coco17", "coco14"]:
        all_images, train_captions_mapping = load_images_coco(train_dataset_configuration)
    # Datasets Flickr30k, Flickr8k_polish, AIDe, Flickr8k  have the same strucutre of files with captions and split informations.
    if train_dataset_configuration["data_name"] in ["flickr30k_polish", "flickr8k_polish", "aide", "flickr8k"]:
        all_images, train_captions_mapping = load_images_flickr(train_dataset_configuration)
    return all_images, train_captions_mapping

In [82]:
import json
import os
from config import general
intersection=dict()
max_set_of_ids=set()
max_dict_of_ids=dict()
min_dict_of_ids=dict()
min_set_of_ids=set()

bleu1_ids=list()
bleu2_ids=list()
bleu3_ids=list()
bleu4_ids=list()
cider_ids=list()
meteor_ids=list()
rouge_ids=list()




def sort_and_get_max_min(metric, name):
    sorted_by=sorted(metric.items(), key=lambda x:x[1])

    sorted_max_n=sorted_by[-1]
    sorted_min_n=sorted_by[1]

    image_id_min=sorted_min_n[0]
    image_id_max=sorted_max_n[0]

    min_set_of_ids.add(image_id_min)
    if image_id_min not in min_dict_of_ids:
        min_dict_of_ids[image_id_min]={"measures":{name},
                                       "dataset_name":dataset_name,
                                       "Bleu_1":results['imgToEval'][image_id_min]['Bleu_1'],
                                       "Bleu_2":results['imgToEval'][image_id_min]['Bleu_2'],
                                        "Bleu_3":results['imgToEval'][image_id_min]['Bleu_3'],
                                        "Bleu_4":results['imgToEval'][image_id_min]['Bleu_4'],
                                        "ROUGE_L":results['imgToEval'][image_id_min]['ROUGE_L'],
                                        "METEOR":results['imgToEval'][image_id_min]['METEOR'],
                                        "CIDEr":results['imgToEval'][image_id_min]['CIDEr'],
                                       "caption":results['imgToEval'][image_id_min]['caption'],
                                       "ground_truth_captions":results['imgToEval'][image_id_min]['ground_truth_captions'],
                                       "result_config_name":result_config_name,
                                       "image_path":all_images_from_split[image_id_min]}
    else:
        a=min_dict_of_ids[image_id_min]["measures"]
        a.add(name)

    max_set_of_ids.add(image_id_max)
    if image_id_max not in max_dict_of_ids:
        max_dict_of_ids[image_id_max]={"measures":{name},
                                       "dataset_name":dataset_name,
                                        "Bleu_1":results['imgToEval'][image_id_max]['Bleu_1'],
                                       "Bleu_2":results['imgToEval'][image_id_max]['Bleu_2'],
                                        "Bleu_3":results['imgToEval'][image_id_max]['Bleu_3'],
                                        "Bleu_4":results['imgToEval'][image_id_max]['Bleu_4'],
                                        "ROUGE_L":results['imgToEval'][image_id_max]['ROUGE_L'],
                                        "METEOR":results['imgToEval'][image_id_max]['METEOR'],
                                        "CIDEr":results['imgToEval'][image_id_max]['CIDEr'],
                                       "caption":results['imgToEval'][image_id_max]['caption'],
                                       "ground_truth_captions":results['imgToEval'][image_id_max]['ground_truth_captions'],
                                       "result_config_name":result_config_name,
                                       "image_path":all_images_from_split[image_id_max]}
    else:
        a=max_dict_of_ids[image_id_max]["measures"]
        a.add(name)
    return image_id_max, image_id_min, sorted_by




for result_config_name in os.listdir(general["results_directory"]):
    if result_config_name.endswith(".json"):
        results = json.load(open("./" + general["results_directory"] + "/" + result_config_name, 'r'))
        dataset_name=results["dataset_name"]

        result_images_ids = list(results['imgToEval'].keys())
        bleu1=dict()
        bleu2=dict()
        bleu3=dict()
        bleu4=dict()
        rouge=dict()
        meteor=dict()
        cider=dict()
        for img_id in result_images_ids:
            bleu1[img_id]=results['imgToEval'][img_id]['Bleu_1']
            bleu2[img_id]=results['imgToEval'][img_id]['Bleu_2']
            bleu3[img_id]=results['imgToEval'][img_id]['Bleu_3']
            bleu4[img_id]=results['imgToEval'][img_id]['Bleu_4']
            rouge[img_id]=results['imgToEval'][img_id]['ROUGE_L']
            meteor[img_id]=results['imgToEval'][img_id]['METEOR']
            cider[img_id]=results['imgToEval'][img_id]['CIDEr']


        all_images_from_split, train_captions = get_images_for_split(dataset_name)
        bleu1_sorted_max_n, bleu1_sorted_min_n, bleu1_sorted=sort_and_get_max_min(bleu1, "Bleu_1")
        bleu2_sorted_max_n, bleu2_sorted_min_n, bleu2_sorted=sort_and_get_max_min(bleu2, "Bleu_2")
        bleu3_sorted_max_n, bleu3_sorted_min_n, bleu3_sorted=sort_and_get_max_min(bleu3,'Bleu_3')
        bleu4_sorted_max_n, bleu4_sorted_min_n, bleu4_sorted=sort_and_get_max_min(bleu4, 'Bleu_4')
        meteor_sorted_max_n, meteor_sorted_min_n, meteor_sorted=sort_and_get_max_min(meteor,'METEOR')
        rouge_sorted_max_n, rouge_sorted_min_n, rouge_sorted=sort_and_get_max_min(rouge, 'ROUGE_L')
        cider_sorted_max_n, cider_sorted_min_n, cider_sorted=sort_and_get_max_min(cider, 'CIDEr')


metrics=['Bleu_4', 'METEOR','ROUGE_L','CIDEr']
for x in max_dict_of_ids.keys():
    max_dict_of_ids[x]["is_best"]=all(item in max_dict_of_ids[x]['measures'] for item in metrics)
for x in min_dict_of_ids.keys():
    min_dict_of_ids[x]["is_best"]=all(item in min_dict_of_ids[x]['measures'] for item in metrics)

print(max_dict_of_ids)
class SetEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, set):
            return list(obj)
        return json.JSONEncoder.default(self, obj)
with open("./" + general["results_directory"] +"/intersection"+ "/intersection_results.json", 'w') as outfile:
    json.dump(
        {"max_dict_of_ids":max_dict_of_ids,
         "min_dict_of_ids":min_dict_of_ids},outfile, cls=SetEncoder)

{'3123463486_f5b36a3624': {'measures': {'Bleu_1'}, 'dataset_name': 'flickr8k', 'Bleu_1': 0.9999999998750002, 'Bleu_2': 0.8451542546153265, 'Bleu_3': 0.4919340733318464, 'Bleu_4': 6.985342055472009e-05, 'ROUGE_L': 0.7331730769230769, 'METEOR': 0.134383761300984, 'CIDEr': 0.9780245030146637, 'caption': 'a black dog is running on the sand', 'ground_truth_captions': ['A brown , black , and white dog runs along on the gravel .', 'A dog runs', 'A little dog running on sand .', 'The brown , white and black dog runs on a gravel surface .', 'The dog is running across the gravel .'], 'result_config_name': 'mixed_flickr8k_flickr8k_resnet50_glove.json', 'image_path': '/home2/data/images/flickr8k/Images/3123463486_f5b36a3624.jpg', 'is_best': False}, '3364026240_645d533fda': {'measures': {'Bleu_2', 'Bleu_3'}, 'dataset_name': 'flickr8k', 'Bleu_1': 0.9999999997500004, 'Bleu_2': 0.9258200995328305, 'Bleu_3': 0.8298265331423224, 'Bleu_4': 0.6914415690877681, 'ROUGE_L': 0.7724312590448625, 'METEOR': 0.12

In [123]:
def get_ngrams(text, n=2, m=None, max_features=20000):
    
    vec = CountVectorizer(ngram_range = (n, n), 
                          max_features = max_features, 
                          stop_words='english').fit(text)
    bag_of_words = vec.transform(text)
    sum_words = bag_of_words.sum(axis = 0) 
    words_freq = [(word, sum_words[0, i]) for word, i in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key = lambda x: x[1], reverse = True)
   
    return words_freq[:m]

In [129]:
def n_gram_train_captions(n, train_captions_dict):
    #     train_captions = set()
#     for val in train_captions_dict.values():
#         for sentence in val:
#             train_captions.add(sentence)

    train_captions = list()
    for val in train_captions_dict.values():
        for sentence in val:
            train_captions.append(sentence)
    print(len(train_captions))
    unigrams = get_ngrams(train_captions)
    print(unigrams)
#     vectorizer = CountVectorizer()
#     X = vectorizer.fit_transform(train_captions)
#     captions_words = vectorizer.get_feature_names()
#     vectorizer2 = CountVectorizer(analyzer='word', ngram_range=(n, n))
#     X2 = vectorizer2.fit_transform(train_captions)
#     captions_grams = vectorizer2.get_feature_names()
#     print(vectorizer2.get_params())
#     return captions_grams



In [130]:
n_gram_train_captions(2, train_captions)

30000
[('white dog', 1087), ('brown dog', 1070), ('black dog', 890), ('dog running', 762), ('little girl', 509), ('young boy', 504), ('black white', 495), ('man wearing', 429), ('little boy', 422), ('young girl', 413), ('dog runs', 392), ('group people', 365), ('dog jumping', 346), ('man woman', 287), ('dog jumps', 281), ('brown white', 260), ('woman wearing', 259), ('wearing red', 255), ('dogs running', 239), ('blue shirt', 230), ('boy wearing', 227), ('running grass', 205), ('young man', 205), ('wearing blue', 200), ('girl wearing', 198), ('white shirt', 198), ('red shirt', 197), ('tan dog', 195), ('wearing black', 195), ('dogs playing', 193), ('soccer ball', 185), ('tennis ball', 178), ('dog playing', 174), ('grassy field', 158), ('man red', 158), ('man black', 151), ('dogs run', 146), ('man blue', 143), ('dogs play', 143), ('swimming pool', 142), ('girl pink', 142), ('man standing', 142), ('children playing', 142), ('wearing white', 141), ('ball mouth', 140), ('small child', 139), 

In [112]:
def n_gram_stat(n, cap):
    caption = [cap]
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(caption)
    captions_words = vectorizer.get_feature_names()
    vectorizer2 = CountVectorizer(analyzer='word', ngram_range=(n, n))
    X2 = vectorizer2.fit_transform(caption)
    captions_grams = vectorizer2.get_feature_names()
    

    return captions_grams, captions_words
    

In [113]:

def show_image_results_captions(image_id, intersection_results):
    """
    Method to show image, ground truth captions, predicted caption and results of metrics
    Parameters
    ----------
    image_id: str
        ID of image
    Returns
    -------
    Prints image, ground truth captions, predicted caption and results of metrics
    """
    #Load results of metrics from file
    image_results = intersection_results[image_id]
    print('Dataset name: {}'.format(image_results["dataset_name"]))
    #Load image
    I = io.imread(image_results['image_path'])
    plt.imshow(I)
    plt.axis('off')
    plt.show()
    print("Ground truth captions")
    print(image_results['ground_truth_captions'])
    print("Predicted captions")
    print(image_results['caption'])
    print(n_gram_stat(2, image_results['caption']))
    #Display results in pretty table
    print( f'\n===== Results =====' )
#     print(image_results["metrics"])
#     print(image_results["is_best"])
    t = PrettyTable(("Bleu_1", "Bleu_2", "Bleu_3", "Bleu_4"))
    t.add_row((image_results["Bleu_1"], image_results["Bleu_2"], image_results["Bleu_3"], image_results["Bleu_4"]))
    t2 = PrettyTable(("METEOR", "ROUGE_L", "CIDEr"))
    t2.add_row((image_results["METEOR"],image_results["ROUGE_L"], image_results["CIDEr"]))
    print(t)
    print(t2)
    print()

In [114]:
#List all files from results directory to view data
type_of_filter = ["max_dict_of_ids", "min_dict_of_ids"]
selectbox = widgets.Select(
    options=type_of_filter,
    value=type_of_filter[1],
    description='Min/max:',
    disabled=False
)

In [115]:
print(selectbox.value)
intersection_results = json.load(open(general["results_directory"] + "/intersection/intersection_results.json"))
inter_results=intersection_results[selectbox.value]
images_ids=list(inter_results.keys())
#Create fancy viever for images, captions and results of evaluation
d=deque(images_ids)
#Button to read image back
left = Button(description="<")
#Button to read next image
right = Button(description=">")

switch = [left, right]

combined = HBox([items for items in switch])
out = Output()

def on_button_left(ex):
    with out:
        clear_output()
        d.rotate(1)
        show_image_results_captions(d[0], inter_results)
def on_button_right(ex):
    with out:
        clear_output()
        d.rotate(-1)
        show_image_results_captions(d[0], inter_results)
l=switch[0].on_click(on_button_left)
r=switch[1].on_click(on_button_right)
display(combined)
display(out)

min_dict_of_ids


Output()